## Cong's ML script for Options Backtesting

- This is a beta version to apply ML to the options data collected since Jan 2022.
- General idea is to use the data downloaded from TDA API to predict future options prices (regression problem).
- Use XGB, LGBM, CatBoost, Random Forest to predict future prices
- Future expansion includes downloading data from other less correlated assets e.g. equities, gold, crypto, bonds etc, clustering before regression!
- File created on 12 Aug 2022

pseudo code
- need to filter off options which have less than 30 days of expiration
- need to create a Y (target) column -> price of option 30 days after quoteTimeinLong, and with about 7 days left in option

#### Change Log:


- 22 Sep 22: predict seems to be working very well. Need to try on real data. Plus, write some code to input unseen data.
- 7 Oct 22: trying to combine the combine.ipynb into this file. df.inTheMoney keeps dropping off for some reason. still can run tho.
- 10 Oct 22: bugs fixed. XGB is working properly.
- 24 Oct 22: introduced XGBClassifier into the picture. code seems to be working fine. need to expand to other files.
- 28 Oct 22: updated 'expected value' formula and trying to backtest now
- 31 Oct 22: v3.1 - 
    - filter out options with mark less than 10
    - predict loss / gain 7 days later
    - sample 10 random trades and see accuracy
    - refine method
    - classifer only, no regressor
    

#### Metadata:
- pc_chg is the percentage difference between the original mark and the predicted mark (for train dataset)
- pred_pc_chg is the percentage difference between the original mark and the predicted mark (for test dataset)
- pc_error is the percentage error between original mark and the predicted mark
- quotetime is the time when the original mark was recorded
- days_to_close is the number of days from open trade to close trade 

## 1a. Import from TDA

In [1]:
import pandas as pd            #opens all CSV files and adds them to dataframe
import glob
import os
from pprint import pprint
import pandas_datareader.data as web
import numpy as np
from datetime import datetime
from time import time
import math
from datetime import timedelta
from datetime import date
pd.set_option('display.max_columns', None)

### CHANGE THESE VARIABLES HERE

In [2]:
# insert df_train here

files = os.path.join('/Users/cyc/Documents/2022 files/_BACKTEST/rstudio-export/_combined files/_combined files', "SPY*.csv")
files = glob.glob(files)
in_tda0 = pd.concat(map(pd.read_csv, files), ignore_index = True).reset_index(drop=True).drop(columns='Unnamed: 0')

# insert df_test here (use this to predict price day_to_close later)
files = os.path.join('/Users/cyc/Documents/2022 files/_BACKTEST/rstudio-export/_combined files/combined files 23 Dec only', "SPY*.csv")      #Importing CSVs from SPY* files
files = glob.glob(files)                                                                                                                     # the above line joins all files with file name beginning with "SPY"
in_tda1 = pd.concat(map(pd.read_csv, files), ignore_index = True).reset_index(drop=True).drop(columns='Unnamed: 0')                                                                         # insert df_test here (use this to predict price day_to_close later)

# insert df with actual mark here
# files = os.path.join('/Users/cyc/Documents/2022 files/_BACKTEST/rstudio-export/_combined files/combined files 8 Nov only', "SPY*.csv")
# files = glob.glob(files)
# in_tda2 = pd.concat(map(pd.read_csv, files), ignore_index = True).reset_index(drop=True).drop(columns='Unnamed: 0')     

days_to_close = 7
prediction_date = '30Dec2022'

In [3]:
in_tda0

,putCall,symbol,description,exchangeName,bid,ask,last,mark,bidSize,askSize,bidAskSize,lastSize,highPrice,lowPrice,openPrice,closePrice,totalVolume,tradeDate,tradeTimeInLong,quoteTimeInLong,netChange,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,theoreticalVolatility,optionDeliverablesList,strikePrice,expirationDate,daysToExpiration,expirationType,lastTradingDay,multiplier,settlementType,deliverableNote,isIndexOption,percentChange,markChange,markPercentChange,intrinsicValue,nonStandard,pennyPilot,inTheMoney,mini
0,CALL,SPY_072022C270,SPY Jul 20 2022 270 Call (Weekly),OPR,125.69,126.08,121.78,125.89,200.0,200.0,200X200,0.0,0.0,0.0,0.0,122.29,0.0,NaN,2022-07-19 19:39:21,2022-07-20 16:18:00,-0.51,270.090,1.0,0.0,-0.057,0.0,0.005,1.0,-4.10,125.890,29.0,NaN,270.0,2022-07-20 20:00:00,0.0,S,2022-07-21 00:00:00,100.0,,NaN,NaN,-0.41,3.60,2.94,125.88,False,True,True,False
1,CALL,SPY_072022C275,SPY Jul 20 2022 275 Call (Weekly),OPR,120.69,121.08,0.00,120.89,200.0,200.0,200X200,0.0,0.0,0.0,0.0,117.29,0.0,NaN,1970-01-01 00:00:00,2022-07-20 16:18:00,0.00,257.902,1.0,0.0,-0.057,0.0,0.005,0.0,0.01,120.890,29.0,NaN,275.0,2022-07-20 20:00:00,0.0,S,2022-07-21 00:00:00,100.0,,NaN,NaN,0.00,3.60,3.07,120.88,False,True,True,False
2,CALL,SPY_072022C280,SPY Jul 20 2022 280 Call (Weekly),OPR,115.69,116.08,0.00,115.89,200.0,200.0,200X200,0.0,0.0,0.0,0.0,112.29,0.0,NaN,1970-01-01 00:00:00,2022-07-20 16:18:00,0.00,245.911,1.0,0.0,-0.056,0.0,0.005,0.0,0.01,115.890,29.0,NaN,280.0,2022-07-20 20:00:00,0.0,S,2022-07-21 00:00:00,100.0,,NaN,NaN,0.00,3.60,3.20,115.88,False,True,True,False
3,CALL,SPY_072022C285,SPY Jul 20 2022 285 Call (Weekly),OPR,110.69,111.08,0.00,110.89,200.0,200.0,200X200,0.0,0.0,0.0,0.0,107.29,0.0,NaN,1970-01-01 00:00:00,2022-07-20 16:18:00,0.00,234.111,1.0,0.0,-0.056,0.0,0.005,0.0,0.01,110.890,29.0,NaN,285.0,2022-07-20 20:00:00,0.0,S,2022-07-21 00:00:00,100.0,,NaN,NaN,0.00,3.60,3.35,110.88,False,True,True,False
4,CALL,SPY_072022C290,SPY Jul 20 2022 290 Call (Weekly),OPR,105.69,106.08,96.69,105.89,200.0,200.0,200X200,0.0,0.0,0.0,0.0,102.29,0.0,NaN,2022-07-18 17:23:41,2022-07-20 16:18:00,-5.60,222.486,1.0,0.0,-0.056,0.0,0.005,100.0,-9.19,105.890,29.0,NaN,290.0,2022-07-20 20:00:00,0.0,S,2022-07-21 00:00:00,100.0,,NaN,NaN,-5.47,3.60,3.52,105.88,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24555915,PUT,SPY_011725P580,SPY Jan 17 2025 580 Put,OPR,214.50,219.50,209.19,217.00,1.0,1.0,1X1,0.0,0.0,0.0,0.0,223.44,0.0,NaN,2022-09-23 13:30:01,2022-10-13 15:46:47,-14.25,19.075,-1.0,0.0,0.000,0.0,0.000,0.0,-8.00,217.165,29.0,NaN,580.0,2025-01-17 21:00:00,827.0,R,2025-01-18 01:00:00,100.0,,NaN,NaN,-6.38,-6.44,-2.88,217.19,False,True,True,False
24555916,PUT,SPY_011725P585,SPY Jan 17 2025 585 Put,OPR,219.57,224.50,0.00,222.04,1.0,1.0,1X1,0.0,0.0,0.0,0.0,228.44,0.0,NaN,1970-01-01 00:00:00,2022-10-13 15:47:01,0.00,18.723,-1.0,0.0,0.000,0.0,0.000,0.0,-0.15,222.165,29.0,NaN,585.0,2025-01-17 21:00:00,827.0,R,2025-01-18 01:00:00,100.0,,NaN,NaN,0.00,-6.41,-2.80,222.19,False,True,True,False
24555917,PUT,SPY_011725P590,SPY Jan 17 2025 590 Put,OPR,224.50,229.50,193.26,227.00,1.0,1.0,1X1,0.0,0.0,0.0,0.0,233.44,0.0,NaN,2022-09-13 16:37:55,2022-10-13 15:46:47,-40.18,18.560,-1.0,0.0,0.000,0.0,0.000,0.0,-33.93,227.165,29.0,NaN,590.0,2025-01-17 21:00:00,827.0,R,2025-01-18 01:00:00,100.0,,NaN,NaN,-17.21,-6.44,-2.76,227.19,False,True,True,False
24555918,PUT,SPY_011725P595,SPY Jan 17 2025 595 Put,OPR,229.50,234.50,0.00,232.00,1.0,1.0,1X1,0.0,0.0,0.0,0.0,238.44,0.0,NaN,1970-01-01 00:00:00,2022-10-13 15:46:47,0.00,19.271,-1.0,0.0,0.000,0.0,0.000,0.0,-0.19,232.165,29.0,NaN,595.0,2025-01-17 21:00:00,827.0,R,2025-01-18 01:00:00,100.0,,NaN,NaN,0.00,-6.44,-2.70,232.19,False,True,True,False


## 1. TRAINING SET

In [4]:
def convert_bool(value):    # converts the column from boolean to numerical - 1 or 0
    if value==True:
        return 1
    else:
        return 0

def prep_df(data):    # processes dataset for further use
    df = data.copy()
    df['inTheMoney'] = df['inTheMoney'].apply(convert_bool)

    df = df.drop(['lastSize', 'openPrice', 'tradeDate',  # drop the unrelevant or columns with no unique values.
                'theoreticalVolatility', 'optionDeliverablesList',
                'multiplier', 'deliverableNote',
                'isIndexOption', 'pennyPilot', 'nonStandard', 'mini'
                ], axis=1)


    # extract stuff from datetime for ML later!
    # df['tradeTimeInLong_year'] = pd.to_datetime(data['tradeTimeInLong']).dt.year
    # df['tradeTimeInLong_month'] = pd.to_datetime(data['tradeTimeInLong']).dt.month
    # df['tradeTimeInLong_day'] = pd.to_datetime(data['tradeTimeInLong']).dt.day
    # df['tradeTimeInLong_dayofweek'] = pd.to_datetime(data['tradeTimeInLong']).dt.dayofweek

    df['quoteTimeInLong_year'] = pd.to_datetime(data['quoteTimeInLong']).dt.year
    df['quoteTimeInLong_month'] = pd.to_datetime(data['quoteTimeInLong']).dt.month
    df['quoteTimeInLong_day'] = pd.to_datetime(data['quoteTimeInLong']).dt.day
    df['quoteTimeInLong_dayofweek'] = pd.to_datetime(data['quoteTimeInLong']).dt.dayofweek

    df['quoteTimeInLong'] = pd.to_datetime(data['quoteTimeInLong']).dt.date

    df['opt_exp_month'] = data.symbol.str[4:6].astype(int)      # parsing symbol to indiv col
    df['opt_exp_day'] = data.symbol.str[6:8].astype(int)
    df['opt_exp_year'] = data.symbol.str[8:10].astype(int)
    df['opt_exp_strike'] = data.symbol.str[11:14].astype(int)
 
    df = df[df['quoteTimeInLong_year'] != 1970]    # drop the rows whose quoteTimeInLong is 1970 (these rows have invalid quotes)

    df = df.groupby(['quoteTimeInLong', 'symbol']).mean().reset_index()             # finding the average mark of each symbol in each respective day

    # days_to_close = 28                                                                # we set the days to close to 28 days here
    df['close_date'] = df['quoteTimeInLong'] + timedelta(days=days_to_close)

    df['putCall'] = df.symbol.str[10]         # extract put or call from option symbol
    putcall_e = pd.get_dummies(df['putCall'], prefix= 'encoded')           # this get_dummies is the same as OneHotEncoder in SKLEARN by alot faster
    df = pd.concat([df, putcall_e], axis=1)

    return df

df = prep_df(in_tda0)
df

,quoteTimeInLong,symbol,bid,ask,last,mark,bidSize,askSize,highPrice,lowPrice,closePrice,totalVolume,netChange,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,strikePrice,daysToExpiration,percentChange,markChange,markPercentChange,intrinsicValue,inTheMoney,quoteTimeInLong_year,quoteTimeInLong_month,quoteTimeInLong_day,quoteTimeInLong_dayofweek,opt_exp_month,opt_exp_day,opt_exp_year,opt_exp_strike,close_date,putCall,encoded_C,encoded_P
0,2022-02-15,SPY_011924C200,243.500000,248.500000,241.87,247.680000,10.000000,10.000000,0.00,0.00,241.54,0.0,0.33,38.315000,0.966,0.0,-0.016,0.504,3.441,54.0,-4.230000,252.431000,200.0,702.0,0.14,6.130000,2.540000,246.100000,1.0,2022.0,2.0,15.0,1.0,1.0,19.0,24.0,200.0,2022-02-22,C,1,0
1,2022-02-15,SPY_011924C210,234.000000,239.000000,243.50,238.280000,10.000000,1.000000,0.00,0.00,232.42,0.0,11.08,37.340000,0.961,0.0,-0.018,0.561,3.577,4.0,7.400000,243.024000,210.0,702.0,4.77,5.870000,2.520000,236.100000,1.0,2022.0,2.0,15.0,1.0,1.0,19.0,24.0,210.0,2022-02-22,C,1,0
2,2022-02-15,SPY_011924C215,229.500000,234.500000,236.43,233.620000,10.000000,10.000000,0.00,0.00,227.86,0.0,8.57,36.831000,0.958,0.0,-0.018,0.590,3.645,1.0,5.330000,238.321000,215.0,702.0,3.76,5.750000,2.520000,231.100000,1.0,2022.0,2.0,15.0,1.0,1.0,19.0,24.0,215.0,2022-02-22,C,1,0
3,2022-02-15,SPY_011924C220,225.000000,230.000000,228.00,228.960000,10.000000,10.000000,0.00,0.00,223.32,0.0,4.68,36.311000,0.956,0.0,-0.019,0.619,3.711,20.0,1.900000,233.618000,220.0,702.0,2.10,5.640000,2.530000,226.100000,1.0,2022.0,2.0,15.0,1.0,1.0,19.0,24.0,220.0,2022-02-22,C,1,0
4,2022-02-15,SPY_011924C225,220.500000,225.500000,221.89,224.320000,10.000000,10.000000,221.89,221.89,218.78,2.0,3.11,35.781000,0.953,0.0,-0.019,0.649,3.777,5.0,0.790000,228.914000,225.0,702.0,1.42,5.550000,2.540000,221.100000,1.0,2022.0,2.0,15.0,1.0,1.0,19.0,24.0,225.0,2022-02-22,C,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703971,2022-12-22,SPY_123022P585,206.578462,206.764615,0.00,206.674615,15.153846,16.461538,0.00,0.00,198.77,0.0,0.00,82.790077,-1.000,0.0,0.000,0.000,0.000,0.0,-0.090000,206.760538,585.0,8.0,0.00,7.897692,3.975385,206.762308,1.0,2022.0,12.0,22.0,3.0,12.0,30.0,22.0,585.0,2022-12-29,P,0,1
1703972,2022-12-22,SPY_123022P590,211.577692,211.762308,194.42,211.673077,12.538462,13.846154,0.00,0.00,203.77,0.0,-9.35,84.270769,-1.000,0.0,0.000,0.000,0.000,0.0,-17.342308,211.760538,590.0,8.0,-4.59,7.896154,3.875385,211.762308,1.0,2022.0,12.0,22.0,3.0,12.0,30.0,22.0,590.0,2022-12-29,P,0,1
1703973,2022-12-22,SPY_123022P595,216.573846,216.760769,0.00,216.669231,13.846154,16.461538,0.00,0.00,208.77,0.0,0.00,85.735846,-1.000,0.0,0.000,0.000,0.000,0.0,-0.094615,216.760538,595.0,8.0,0.00,7.894615,3.782308,216.762308,1.0,2022.0,12.0,22.0,3.0,12.0,30.0,22.0,595.0,2022-12-29,P,0,1
1703974,2022-12-22,SPY_123022P600,221.572308,221.757692,219.82,221.666923,15.153846,15.153846,0.00,0.00,213.77,0.0,6.05,87.187154,-1.000,0.0,0.000,0.000,0.000,0.0,-1.942308,221.760538,600.0,8.0,2.83,7.892308,3.693846,221.762308,1.0,2022.0,12.0,22.0,3.0,12.0,30.0,22.0,600.0,2022-12-29,P,0,1


In [5]:
# THIS IS FOR THE TRAIN SET. 

df7 = df[['quoteTimeInLong', 'symbol', 'mark']].copy()             # df28 is a 2nd table i created to find the price of the option 28 days later
df1 = pd.merge(df, df7, how='left', left_on=['close_date','symbol'], right_on=['quoteTimeInLong','symbol'])

# print('Non-null values in mark_y:', new_df.mark_y.notnull().sum())
# print('Null values in mark_y:', new_df.mark_y.isnull().sum())              # 500k null values of mark_y, which we need to get rid of. Null values exist for mark 28 days later either cos there is no data or the option expired before 28 days

df2 = df1.dropna(subset='mark_y')                        # drop null values of mark_y 

# df_train = df2.drop(['lastSize', 'openPrice', 'tradeDate',  # drop the unrelevant or columns with no unique values.
#                 'theoreticalVolatility', 'optionDeliverablesList',
#                 'multiplier', 'deliverableNote',
#                 'isIndexOption', 'pennyPilot', 'nonStandard', 'mini',
#                 'symbol', 'putCall', 'quoteTimeInLong_x',
#                 'close_date', 'quoteTimeInLong_y'
#                 ], axis=1)

df_train = df2.drop(['quoteTimeInLong_x',
                'close_date', 'quoteTimeInLong_y', 'putCall','symbol'
                ], axis=1)

df_train = df_train[df_train['daysToExpiration'] >= days_to_close]     # drop values with daystoexpiration less than 28 days

In [6]:
df_train

,bid,ask,last,mark_x,bidSize,askSize,highPrice,lowPrice,closePrice,totalVolume,netChange,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,strikePrice,daysToExpiration,percentChange,markChange,markPercentChange,intrinsicValue,inTheMoney,quoteTimeInLong_year,quoteTimeInLong_month,quoteTimeInLong_day,quoteTimeInLong_dayofweek,opt_exp_month,opt_exp_day,opt_exp_year,opt_exp_strike,encoded_C,encoded_P,mark_y
10140,0.081000,0.199000,0.22,0.141000,1015.300000,943.600000,0.00,0.00,0.16,0.0,0.06,16.815000,0.00600,0.000,-0.002000,0.077600,0.024900,1165.0,0.220000,0.143200,675.0,337.8,37.24,-0.021000,-10.468000,-229.109000,0.0,2022.0,2.0,16.0,2.0,1.0,20.0,23.0,675.0,1,0,0.130000
10146,0.039000,0.150000,0.15,0.099000,920.400000,162.300000,0.12,0.12,0.10,3.2,0.05,17.653400,0.00480,0.000,-0.001000,0.056000,0.017700,26.0,0.150000,0.100100,705.0,337.8,51.36,-0.001000,1.092000,-259.109000,0.0,2022.0,2.0,16.0,2.0,1.0,20.0,23.0,705.0,1,0,0.080000
19284,3.356000,4.872000,4.54,3.834000,1.000000,40.200000,0.00,0.00,3.86,0.0,0.68,16.330700,0.07690,0.001,-0.009000,1.091900,0.869300,355.0,4.540000,3.831600,700.0,1037.8,17.49,-0.034000,-0.858000,-254.109000,0.0,2022.0,2.0,16.0,2.0,12.0,20.0,24.0,700.0,1,0,3.510000
19286,1.400000,4.000000,3.20,3.212000,20.100000,1.000000,0.00,0.00,3.52,0.0,-0.32,16.115300,0.06720,0.001,-0.007800,0.977600,0.757700,93.0,3.200000,3.215200,710.0,1037.8,-9.12,-0.308000,-8.696000,-264.109000,0.0,2022.0,2.0,16.0,2.0,12.0,20.0,24.0,710.0,1,0,2.450000
19700,239.250000,244.125000,241.87,241.687500,5.750000,5.750000,0.00,0.00,248.48,0.0,-6.61,37.997250,0.96825,0.000,-0.017750,0.504250,3.430875,54.0,1.210000,248.011875,200.0,701.0,-2.66,-6.792500,-2.735000,240.660000,1.0,2022.0,2.0,17.0,3.0,1.0,19.0,24.0,200.0,1,0,224.510769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666423,196.703846,196.925385,0.00,196.815385,9.230769,8.615385,0.00,0.00,187.28,0.0,0.00,58.562500,-0.99500,0.000,-0.044000,0.000462,-0.251000,0.0,1.636923,197.306000,585.0,15.0,0.00,9.533077,5.090000,195.177692,1.0,2022.0,12.0,15.0,3.0,12.0,30.0,22.0,585.0,0,1,206.674615
1666424,201.702308,201.923077,194.42,201.814615,9.230769,8.000000,0.00,0.00,192.28,0.0,2.14,59.353455,-0.99500,0.000,-0.044000,0.001000,-0.253000,0.0,-5.757692,201.389091,590.0,15.0,1.11,9.530769,4.956154,200.177692,1.0,2022.0,12.0,15.0,3.0,12.0,30.0,22.0,590.0,0,1,211.673077
1666425,206.701538,206.923077,0.00,206.814615,9.230769,8.000000,0.00,0.00,197.28,0.0,0.00,60.260846,-0.99500,0.000,-0.043000,0.001000,-0.255000,0.0,1.635385,205.834462,595.0,15.0,0.00,9.530000,4.830769,205.177692,1.0,2022.0,12.0,15.0,3.0,12.0,30.0,22.0,595.0,0,1,216.669231
1666426,211.697692,211.919231,231.00,211.810769,9.230769,8.000000,0.00,0.00,202.28,0.0,28.72,61.344231,-0.99500,0.000,-0.042615,0.001000,-0.257000,0.0,20.822308,210.824308,600.0,15.0,14.20,9.526154,4.710000,210.177692,1.0,2022.0,12.0,15.0,3.0,12.0,30.0,22.0,600.0,0,1,221.666923


In [7]:
# THIS IS FOR THE TEST SET

df3 = prep_df(in_tda1)
df_test = df3.drop([ 
                'symbol', 'putCall', 'close_date', 
                'quoteTimeInLong'
                # , 'inTheMoney'
                ], axis=1)



df_test = df_test[df_test['daysToExpiration'] >= days_to_close]    # drop rows with daystoexpiration less than 28 days

In [8]:
df_test

,bid,ask,last,mark,bidSize,askSize,highPrice,lowPrice,closePrice,totalVolume,netChange,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,strikePrice,daysToExpiration,percentChange,markChange,markPercentChange,intrinsicValue,inTheMoney,quoteTimeInLong_year,quoteTimeInLong_month,quoteTimeInLong_day,quoteTimeInLong_dayofweek,opt_exp_month,opt_exp_day,opt_exp_year,opt_exp_strike,encoded_C,encoded_P
0,71.741538,71.942308,0.00,71.843077,11.923077,10.615385,0.0,0.0,71.15,0.0,0.00,42.140000,0.997000,0.000,-0.033,0.004615,0.099,0.0,0.319231,70.651500,310.0,11.0,0.00,0.690000,0.965385,71.523077,1.0,2022.0,12.0,23.0,4.0,1.0,3.0,23.0,310.0,1,0
1,61.763077,61.966154,57.30,61.866154,10.615385,10.615385,0.0,0.0,61.19,0.0,-3.89,NaN,NaN,NaN,NaN,0.007077,NaN,15.0,-4.223846,NaN,320.0,11.0,-6.35,0.676923,1.107692,61.523077,1.0,2022.0,12.0,23.0,4.0,1.0,3.0,23.0,320.0,1,0
2,56.775385,56.977692,0.00,56.880000,11.923077,10.615385,0.0,0.0,56.20,0.0,0.00,NaN,NaN,NaN,NaN,0.009923,NaN,0.0,0.354615,NaN,325.0,11.0,0.00,0.673077,1.196154,56.523077,1.0,2022.0,12.0,23.0,4.0,1.0,3.0,23.0,325.0,1,0
3,55.778462,55.981538,0.00,55.881538,10.615385,10.615385,0.0,0.0,55.21,0.0,0.00,35.196000,0.993000,0.001,-0.047,0.010077,0.103,0.0,0.356154,53.828000,326.0,11.0,0.00,0.671538,1.219231,55.523077,1.0,2022.0,12.0,23.0,4.0,1.0,3.0,23.0,326.0,1,0
4,54.780000,54.985385,0.00,54.884615,10.615385,13.230769,0.0,0.0,54.21,0.0,0.00,34.877667,0.992667,0.001,-0.045,0.011615,0.104,0.0,0.358462,53.414667,327.0,11.0,0.00,0.670000,1.241538,54.523077,1.0,2022.0,12.0,23.0,4.0,1.0,3.0,23.0,327.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7245,203.367692,203.544615,0.00,203.458462,15.846154,14.538462,0.0,0.0,204.28,0.0,0.00,86.394923,-1.000000,0.000,0.000,0.000000,0.000,0.0,-0.020000,203.490923,585.0,7.0,0.00,-0.821538,-0.403846,203.476154,1.0,2022.0,12.0,23.0,4.0,12.0,30.0,22.0,585.0,0,1
7246,208.366923,208.542308,194.42,208.457692,14.538462,11.923077,0.0,0.0,209.28,0.0,-14.86,87.971231,-1.000000,0.000,0.000,0.000000,0.000,0.0,-14.056923,208.490923,590.0,7.0,-7.10,-0.822308,-0.393846,208.476154,1.0,2022.0,12.0,23.0,4.0,12.0,30.0,22.0,590.0,0,1
7247,213.366154,213.542308,0.00,213.457692,14.538462,17.153846,0.0,0.0,214.28,0.0,0.00,89.530615,-1.000000,0.000,0.000,0.000000,0.000,0.0,-0.021538,213.490923,595.0,7.0,0.00,-0.822308,-0.386154,213.476154,1.0,2022.0,12.0,23.0,4.0,12.0,30.0,22.0,595.0,0,1
7248,218.366154,218.542308,219.82,218.457692,14.538462,17.153846,0.0,0.0,219.28,0.0,0.54,91.075000,-1.000000,0.000,0.000,0.000000,0.000,0.0,1.343846,218.490923,600.0,7.0,0.25,-0.822308,-0.377692,218.476154,1.0,2022.0,12.0,23.0,4.0,12.0,30.0,22.0,600.0,0,1


In [9]:
# df_actual = prep_df(in_tda2)

# df_actual = df_actual[['opt_exp_month', 'opt_exp_day', 'opt_exp_year', 'opt_exp_strike', 'encoded_C', 'encoded_P', 'mark']]
# df_actual = df_actual.rename(columns={'mark': 'mark_actual'})

# df_actual

## 2 Encoding

In [10]:
# calculating RMSE & R2 for using purely the train dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
from catboost import CatBoostClassifier
from sklearn import metrics

df_train['outcome'] = np.where(df_train['mark_x'] > df_train['mark_y'], 0, 1)

X = df_train.drop(['mark_y', 'outcome'], axis=1)
y = df_train['outcome']

X_test = df_test.rename(columns= {'mark':'mark_x'})

reg = CatBoostClassifier()
reg.fit(X, y)      # fit model with X and y instead of X_train and y_train
y_pred = reg.predict(X_test)   # use fresh 23 Sep 2022 file as X_test

Learning rate set to 0.215805
0:	learn: 0.6601214	total: 152ms	remaining: 2m 31s
1:	learn: 0.6413288	total: 222ms	remaining: 1m 50s
2:	learn: 0.6283325	total: 282ms	remaining: 1m 33s
3:	learn: 0.6197508	total: 341ms	remaining: 1m 24s
4:	learn: 0.6101473	total: 396ms	remaining: 1m 18s
5:	learn: 0.5874438	total: 469ms	remaining: 1m 17s
6:	learn: 0.5788916	total: 530ms	remaining: 1m 15s
7:	learn: 0.5632833	total: 596ms	remaining: 1m 13s
8:	learn: 0.5540710	total: 657ms	remaining: 1m 12s
9:	learn: 0.5298933	total: 722ms	remaining: 1m 11s
10:	learn: 0.4999762	total: 787ms	remaining: 1m 10s
11:	learn: 0.4927592	total: 845ms	remaining: 1m 9s
12:	learn: 0.4701135	total: 910ms	remaining: 1m 9s
13:	learn: 0.4506368	total: 979ms	remaining: 1m 8s
14:	learn: 0.4392756	total: 1.05s	remaining: 1m 8s
15:	learn: 0.4318493	total: 1.12s	remaining: 1m 8s
16:	learn: 0.4248231	total: 1.19s	remaining: 1m 9s
17:	learn: 0.4151697	total: 1.25s	remaining: 1m 8s
18:	learn: 0.4049845	total: 1.32s	remaining: 1m 8s


In [11]:
combi = pd.concat([X_test.reset_index(drop=True),
                    pd.Series(y_pred).reset_index(drop=True),
                    # pd.Series(df_actual['mark']).reset_index(drop=True)
                    ], 
                    axis=1)
combi = combi.rename(columns={0: 'y_pred_binary'})

combi

,bid,ask,last,mark_x,bidSize,askSize,highPrice,lowPrice,closePrice,totalVolume,netChange,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,strikePrice,daysToExpiration,percentChange,markChange,markPercentChange,intrinsicValue,inTheMoney,quoteTimeInLong_year,quoteTimeInLong_month,quoteTimeInLong_day,quoteTimeInLong_dayofweek,opt_exp_month,opt_exp_day,opt_exp_year,opt_exp_strike,encoded_C,encoded_P,y_pred_binary
0,71.741538,71.942308,0.00,71.843077,11.923077,10.615385,0.0,0.0,71.15,0.0,0.00,42.140000,0.997000,0.000,-0.033,0.004615,0.099,0.0,0.319231,70.651500,310.0,11.0,0.00,0.690000,0.965385,71.523077,1.0,2022.0,12.0,23.0,4.0,1.0,3.0,23.0,310.0,1,0,0
1,61.763077,61.966154,57.30,61.866154,10.615385,10.615385,0.0,0.0,61.19,0.0,-3.89,NaN,NaN,NaN,NaN,0.007077,NaN,15.0,-4.223846,NaN,320.0,11.0,-6.35,0.676923,1.107692,61.523077,1.0,2022.0,12.0,23.0,4.0,1.0,3.0,23.0,320.0,1,0,1
2,56.775385,56.977692,0.00,56.880000,11.923077,10.615385,0.0,0.0,56.20,0.0,0.00,NaN,NaN,NaN,NaN,0.009923,NaN,0.0,0.354615,NaN,325.0,11.0,0.00,0.673077,1.196154,56.523077,1.0,2022.0,12.0,23.0,4.0,1.0,3.0,23.0,325.0,1,0,1
3,55.778462,55.981538,0.00,55.881538,10.615385,10.615385,0.0,0.0,55.21,0.0,0.00,35.196000,0.993000,0.001,-0.047,0.010077,0.103,0.0,0.356154,53.828000,326.0,11.0,0.00,0.671538,1.219231,55.523077,1.0,2022.0,12.0,23.0,4.0,1.0,3.0,23.0,326.0,1,0,0
4,54.780000,54.985385,0.00,54.884615,10.615385,13.230769,0.0,0.0,54.21,0.0,0.00,34.877667,0.992667,0.001,-0.045,0.011615,0.104,0.0,0.358462,53.414667,327.0,11.0,0.00,0.670000,1.241538,54.523077,1.0,2022.0,12.0,23.0,4.0,1.0,3.0,23.0,327.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6153,203.367692,203.544615,0.00,203.458462,15.846154,14.538462,0.0,0.0,204.28,0.0,0.00,86.394923,-1.000000,0.000,0.000,0.000000,0.000,0.0,-0.020000,203.490923,585.0,7.0,0.00,-0.821538,-0.403846,203.476154,1.0,2022.0,12.0,23.0,4.0,12.0,30.0,22.0,585.0,0,1,1
6154,208.366923,208.542308,194.42,208.457692,14.538462,11.923077,0.0,0.0,209.28,0.0,-14.86,87.971231,-1.000000,0.000,0.000,0.000000,0.000,0.0,-14.056923,208.490923,590.0,7.0,-7.10,-0.822308,-0.393846,208.476154,1.0,2022.0,12.0,23.0,4.0,12.0,30.0,22.0,590.0,0,1,1
6155,213.366154,213.542308,0.00,213.457692,14.538462,17.153846,0.0,0.0,214.28,0.0,0.00,89.530615,-1.000000,0.000,0.000,0.000000,0.000,0.0,-0.021538,213.490923,595.0,7.0,0.00,-0.822308,-0.386154,213.476154,1.0,2022.0,12.0,23.0,4.0,12.0,30.0,22.0,595.0,0,1,1
6156,218.366154,218.542308,219.82,218.457692,14.538462,17.153846,0.0,0.0,219.28,0.0,0.54,91.075000,-1.000000,0.000,0.000,0.000000,0.000,0.0,1.343846,218.490923,600.0,7.0,0.25,-0.822308,-0.377692,218.476154,1.0,2022.0,12.0,23.0,4.0,12.0,30.0,22.0,600.0,0,1,1


In [12]:
y_pred

array([0, 1, 1, ..., 1, 1, 1])

In [13]:
# note, df_actual has 8150 rows compared to 7000+ of df_test. hence, we cant just concat directly and need to perform a merge.

# combi2 = pd.merge(combi, df_actual, how='left', 
#                 left_on=['opt_exp_month', 'opt_exp_day', 'opt_exp_year', 'opt_exp_strike', 'encoded_C', 'encoded_P'],
#                 right_on=['opt_exp_month', 'opt_exp_day', 'opt_exp_year', 'opt_exp_strike', 'encoded_C', 'encoded_P'])
# combi2

In [14]:
# combi2['y_outcome'] = np.where(combi2['mark_x'] > combi2['mark_actual'], 0, 1)
# combi2['pred_istrue'] = combi2['y_pred_binary'] == combi2['y_outcome']
# combi2

In [15]:
# print('%age of predictions that are wrong:', (combi2['y_outcome'] != combi2['y_pred_binary']).sum() / combi2.shape[0] * 100)

## 2. Predictions!

In [16]:
filter = combi[['opt_exp_month', 'opt_exp_day','opt_exp_year', 
        'opt_exp_strike','encoded_C', 'encoded_P', 
        'mark_x','y_pred_binary']]

        # filter = combi2[['opt_exp_month', 'opt_exp_day','opt_exp_year', 
        # 'opt_exp_strike','encoded_C', 'encoded_P', 
        # 'mark_x','mark_actual','y_pred_binary','y_outcome', 'pred_istrue']]

out = filter[(filter['mark_x'] >= 10) & (filter['mark_x'] < 30)].sample(20)
out

,opt_exp_month,opt_exp_day,opt_exp_year,opt_exp_strike,encoded_C,encoded_P,mark_x,y_pred_binary
2866,3.0,17.0,23.0,393.0,1,0,11.910769,0
2121,2.0,3.0,23.0,378.0,1,0,14.185385,0
3431,3.0,31.0,23.0,382.0,1,0,19.257692,0
3421,3.0,31.0,23.0,372.0,1,0,25.490000,0
3089,3.0,17.0,23.0,381.0,0,1,14.659231,1
4600,6.0,30.0,23.0,406.0,1,0,15.980000,0
6027,12.0,30.0,22.0,410.0,0,1,28.480000,1
3590,3.0,31.0,23.0,376.0,0,1,14.109231,1
202,1.0,3.0,23.0,404.0,0,1,22.483077,1
4030,6.0,16.0,23.0,403.0,1,0,16.500769,0


In [17]:
out.to_csv('catb predict of'+ prediction_date +'_v3.1 fwd.csv')

In [18]:
out

,opt_exp_month,opt_exp_day,opt_exp_year,opt_exp_strike,encoded_C,encoded_P,mark_x,y_pred_binary
2866,3.0,17.0,23.0,393.0,1,0,11.910769,0
2121,2.0,3.0,23.0,378.0,1,0,14.185385,0
3431,3.0,31.0,23.0,382.0,1,0,19.257692,0
3421,3.0,31.0,23.0,372.0,1,0,25.490000,0
3089,3.0,17.0,23.0,381.0,0,1,14.659231,1
4600,6.0,30.0,23.0,406.0,1,0,15.980000,0
6027,12.0,30.0,22.0,410.0,0,1,28.480000,1
3590,3.0,31.0,23.0,376.0,0,1,14.109231,1
202,1.0,3.0,23.0,404.0,0,1,22.483077,1
4030,6.0,16.0,23.0,403.0,1,0,16.500769,0


In [19]:
import subprocess

applescript = """
display dialog "Run complete!" ¬
with title "This is a pop-up window" ¬
with icon caution ¬
buttons {"OK"}
"""

subprocess.call("osascript -e '{}'".format(applescript), shell=True)

button returned:OK


0